In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 50
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.125      0.         0.11111111 0.         0.
  0.14285714 0.14285714 0.         0.         0.         0.
  0.         0.         0.         0.125      0.         0.
  0.         0.14285714]
 [0.125      0.         0.125      0.11111111 0.         0.125
  0.         0.125      0.125      0.         0.         0.125
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.125      0.         0.         0.         0.14285714
  0.         0.         0.         0.11111111 0.11111111 0.
  0.16666667 0.         0.         0.         0.         0.
  0.         0.        ]
 [0.11111111 0.11111111 0.         0.         0.11111111 0.11111111
  0.         0.11111111 0.11111111 0.11111111 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.11111111]
 [0.         0.         0.         0.11111111 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.1

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[6, 6, 7, 5, 8, 2, 6, 6, 6, 5, 8, 8, 5, 5, 6, 5, 7, 6, 6, 7]
1
4
14
16
18
19


In [6]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.size)
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split





X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


X, y = random_split(X_train, y_train, no_users, 1234)

In [7]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0, 3.0, 3.0])
W2 = np.array([-2.0, 2.0, 3.0, -3.0])
W3 = 2 * W1
W4 = 2  * W2
W = [W1, W2]
m = 200
n = 4

scaler = [1.0, -1.0]

noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i ]) + np.random.normal(0,noise_sd)
        data = X[count]
        data[:, 0:4] *= scaler[i]
        datapoints[count] = {
                'features': data,
                'degree': node_degree(count, G),
                'label': y[count],
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [9]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(9, 4, bias=False)
        self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        #output = self.fc3(x)
        return output

In [10]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [11]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        #new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        #vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(48.1133, grad_fn=<MseLossBackward0>) tensor([-0.1712,  0.1184,  0.0315,  0.1727, -0.1456, -0.1224, -0.1589, -0.0821,
        -0.2563, -0.3594,  0.4645, -0.7706,  0.5060, -0.5884, -0.5278, -0.5631,
         0.1299,  0.1909,  0.8116, -0.5697, -0.4041, -0.5035,  0.3225,  0.1931,
         0.4018,  0.3837,  1.4779,  0.1164, -0.1162, -0.2501,  0.3692, -0.3315,
        -0.2943, -0.3497, -0.2767, -0.0494, -5.3704, -2.2038, -3.2683, -2.8022]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(53.3068, grad_fn=<MseLossBackwar

3 tensor(52.0993, grad_fn=<MseLossBackward0>) tensor([-2.0076e-01,  1.0459e-01, -6.2969e-04,  2.6472e-01, -2.5556e-01,
        -1.8682e-01, -2.6294e-01, -9.4497e-02, -1.0981e-01, -1.1964e+00,
         9.6917e-01, -7.7178e-01,  7.0624e-01, -8.1397e-01, -1.6968e-01,
        -7.2543e-01, -6.4447e-02,  2.4457e-01,  5.7671e-01, -2.7516e-01,
        -1.0725e-01, -1.1363e+00,  9.7476e-01,  5.7101e-01,  9.5354e-01,
         5.0487e-01,  8.7981e-01, -1.7787e-01,  4.9626e-02, -5.1159e-01,
         6.5561e-01, -4.5915e-01, -2.3576e-01, -4.5066e-01, -5.7723e-01,
        -4.4347e-02, -6.4704e+00, -3.1280e+00, -3.0150e+00, -4.2302e+00]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.04

        -0.1872, -0.2618, -0.7427,  0.0348, -4.9809, -1.9301, -2.9307, -3.8241]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
5 tensor(56.9826, grad_fn=<MseLossBackward0>) tensor([-0.2296,  0.1556,  0.2467,  0.1827, -0.1728, -0.1181, -0.2050, -0.1072,
        -0.1282, -0.8052,  0.6589,  0.1511,  0.6946, -0.8982, -0.7385, -0.9998,
        -0.1878,  0.0857,  0.8678, -0.5003, -1.0721, -0.5816,  0.5989,  0.0631,
         0.7116,  0.6026,  1.0276,  0.0905, -0.1734, -0.3006,  0.4082, -0.2120,
        -0.2488, -0.2911, -0.6767,  0.1957, -6.8923, -1.6509, -3.9710, -3.2423]) 

8 tensor(72.3950, grad_fn=<MseLossBackward0>) tensor([-2.5430e-01,  1.8072e-01,  1.2652e-01,  3.2804e-01, -2.4440e-01,
        -1.3928e-01, -3.0307e-01, -2.7703e-01, -1.5976e-01, -8.8077e-01,
         6.6133e-01, -2.2503e-01,  3.9601e-01, -6.7702e-01, -4.1408e-01,
        -8.6414e-01,  7.2213e-03, -2.1133e-01,  8.9880e-01, -5.7593e-01,
        -4.9425e-01, -1.3916e+00,  9.4598e-01,  4.9105e-01,  1.2200e+00,
         1.4380e+00,  1.0311e+00, -2.5709e-01,  1.4907e-01, -3.5525e-01,
         6.1660e-01, -3.0981e-01, -1.9732e-01, -4.1752e-01, -9.3903e-01,
        -7.8445e-02, -8.2894e+00, -2.0293e+00, -4.4581e+00, -3.6872e+00]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.04

torch.Size([50, 1])
torch.Size([50, 1])
10 tensor(56.9826, grad_fn=<MseLossBackward0>) tensor([-0.2296,  0.1556,  0.2467,  0.1827, -0.1728, -0.1181, -0.2050, -0.1072,
        -0.1282, -0.8052,  0.6589,  0.1511,  0.6946, -0.8982, -0.7385, -0.9998,
        -0.1878,  0.0857,  0.8678, -0.5003, -1.0721, -0.5816,  0.5989,  0.0631,
         0.7116,  0.6026,  1.0276,  0.0905, -0.1734, -0.3006,  0.4082, -0.2120,
        -0.2488, -0.2911, -0.6767,  0.1957, -6.8923, -1.6509, -3.9710, -3.2423]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
10 tensor(54.3241, grad_fn=<MseLossBackw

...

94 tensor(49.8704, grad_fn=<MseLossBackward0>) tensor([-0.1427,  0.1122,  0.0664,  0.1705, -0.1715, -0.1133, -0.1732, -0.0482,
        -0.1107, -0.2952,  0.4130, -0.3310,  1.0071, -1.1160, -0.6771, -1.0591,
         0.0706,  0.4049,  0.5746, -0.3155, -0.3438, -0.2799,  0.2964, -0.3677,
         0.1790,  0.5955,  1.6424,  0.2445, -0.2234, -0.2931,  0.3959, -0.4678,
        -0.2697, -0.4049, -0.1906, -0.1298, -4.9053, -2.2044, -3.4560, -3.2506]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
94 tensor(47.8869, grad_fn=<MseLossBackward0>) tensor([-0.1568,  0.0857,  0.1464

95 tensor(44.9354, grad_fn=<MseLossBackward0>) tensor([-0.0786,  0.0442,  0.0254,  0.2368, -0.2325, -0.1483, -0.2400, -0.0375,
        -0.1398, -1.0750,  0.9824, -0.6112,  0.6428, -0.6682, -0.2831, -0.6963,
        -0.1282,  0.0679,  0.4449, -0.3694, -0.2023, -0.6868,  0.8514,  0.2821,
         0.8533, -0.2469,  0.7725,  0.0724, -0.0906, -0.3808,  0.5113, -0.3963,
        -0.3242, -0.4304, -0.3080, -0.0274, -5.4515, -2.5752, -2.8122, -3.5703]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
95 tensor(38.5651, grad_fn=<MseLossBackward0>) tensor([-0.1166,  0.1437, -0.1759, 

        -0.2618, -0.3508, -0.4526,  0.0705, -5.1247, -2.1413, -3.4694, -2.7520]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
97 tensor(52.0993, grad_fn=<MseLossBackward0>) tensor([-2.0076e-01,  1.0459e-01, -6.2969e-04,  2.6472e-01, -2.5556e-01,
        -1.8682e-01, -2.6294e-01, -9.4497e-02, -1.0981e-01, -1.1964e+00,
         9.6917e-01, -7.7178e-01,  7.0624e-01, -8.1397e-01, -1.6968e-01,
        -7.2543e-01, -6.4447e-02,  2.4457e-01,  5.7671e-01, -2.7516e-01,
        -1.0725e-01, -1.1363e+00,  9.7476e-01,  5.7101e-01,  9.5354e-01,
         5.0487e-01,  8.7981e-01, -

98 tensor(72.3950, grad_fn=<MseLossBackward0>) tensor([-2.5430e-01,  1.8072e-01,  1.2652e-01,  3.2804e-01, -2.4440e-01,
        -1.3928e-01, -3.0307e-01, -2.7703e-01, -1.5976e-01, -8.8077e-01,
         6.6133e-01, -2.2503e-01,  3.9601e-01, -6.7702e-01, -4.1408e-01,
        -8.6414e-01,  7.2213e-03, -2.1133e-01,  8.9880e-01, -5.7593e-01,
        -4.9425e-01, -1.3916e+00,  9.4598e-01,  4.9105e-01,  1.2200e+00,
         1.4380e+00,  1.0311e+00, -2.5709e-01,  1.4907e-01, -3.5525e-01,
         6.1660e-01, -3.0981e-01, -1.9732e-01, -4.1752e-01, -9.3903e-01,
        -7.8445e-02, -8.2894e+00, -2.0293e+00, -4.4581e+00, -3.6872e+00]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0

        -0.2400, -0.3093, -0.1061, -0.1661, -6.0437, -1.8537, -3.6758, -2.5305]) tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(53.6040, grad_fn=<MseLossBackward0>) tensor([-0.0851,  0.0385,  0.0722,  0.2075, -0.1525, -0.1163, -0.1715, -0.1716,
        -0.1672, -0.2695,  0.3893, -0.9661,  0.5759, -0.4012, -0.2815, -0.3967,
        -0.1076,  0.3917,  0.5674, -0.4016, -0.4106, -0.4963,  0.2445, -0.0693,
         0.2442,  0.7905,  1.2199,  0.2028, -0.2496, -0.3884,  0.4438, -0.2437,
        -0.1872, -0.2618, -0.7427,  0.0348, -4.9809, -1.9301, -2.9307, -3.8241])

In [12]:
parameters_to_vector(model.parameters())

tensor([ 0.2235, -0.0973, -0.2852, -0.2029,  0.2218,  0.2281,  0.2715,  0.0543,
         0.2085,  0.1030, -0.2429,  0.2934,  0.2813,  0.2645, -0.0455,  0.2404,
        -0.2080, -0.2315,  0.0014, -0.1183, -0.1677, -0.0731,  0.0520, -0.2141,
         0.3324,  0.0407,  0.2600, -0.1340,  0.0254,  0.2640, -0.2909,  0.2186,
         0.1556, -0.1506,  0.1190,  0.0011,  0.0475,  0.2595, -0.2087,  0.1037],
       grad_fn=<CatBackward0>)

In [13]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [14]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3
                                   )

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [15]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [16]:
print(projection_list[0])

[0, tensor([[1.9948, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.9948, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.9948,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.9948, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.9948, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9948]]), 0, 0, tensor([[1.1173, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1173, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.1173,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.1173, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.1173, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.1173]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, tensor([[1.1013, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1013, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.1013,  ...

In [17]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [18]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 0.3054, -0.2010, -0.1876, -0.0028,  0.2577,  0.1404,  0.0177,  0.1863,
        -0.3272,  0.1281, -0.1372,  0.2988, -0.2581, -0.2330,  0.3324,  0.0934,
         0.0263,  0.1237,  0.2444, -0.1767,  0.1775,  0.3122,  0.1055, -0.1585,
         0.0680, -0.3066,  0.3181,  0.2193,  0.0477, -0.0470, -0.2661,  0.1525,
         0.1462,  0.2871,  0.1531, -0.2106,  0.2963,  0.1908, -0.1184, -0.1955])


tensor([ 0.6109, -0.4020, -0.3753, -0.0055,  0.5155,  0.2807,  0.0354,  0.3726,
        -0.6543,  0.2561, -0.2744,  0.5975, -0.5162, -0.4660,  0.6647,  0.1868,
         0.0526,  0.2474,  0.4887, -0.3534,  0.3551,  0.6245,  0.2110, -0.3171,
         0.1361, -0.6132,  0.6363,  0.4387,  0.0955, -0.0940, -0.5323,  0.3050,
         0.2925,  0.5741,  0.3063, -0.4212,  0.5927,  0.3816, -0.2369, -0.3910],
       grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/2000 [00:00<13:37,  2.44it/s]

Training_loss 52.32741


  0%|          | 2/2000 [00:00<10:47,  3.09it/s]

Training_loss 52.23904


  0%|          | 3/2000 [00:01<12:18,  2.70it/s]

...

Training_loss 5.39717


100%|█████████▉| 1999/2000 [16:17<00:00,  2.12it/s]

Training_loss 5.39347


100%|██████████| 2000/2000 [16:18<00:00,  2.04it/s]

Training_loss 5.39559


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([ 0.2975,  0.2338,  0.3959,  0.4531, -0.2543, -0.5216, -0.2689,  0.0679,
         0.0881, -0.4987,  0.9672,  0.2327,  0.1979,  0.2809,  0.0547,  0.2902,
        -0.1267, -0.5663,  1.1430, -0.4069,  0.1269, -0.0773,  0.3289, -0.1364,
         0.3826,  0.7262, -0.0390,  0.0828,  1.0114, -0.9897,  0.1473,  0.5010,
        -0.5570,  0.5004,  1.1288,  0.2513,  1.8504,  1.7718,  2.0736,  2.6678],
       grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

1
4
14
16
18
19


In [23]:
parameters_to_vector(models[0].parameters())

tensor([ 0.4526, -1.0551,  0.1519, -0.1993, -0.0724, -0.4891,  0.0398,  0.4347,
        -0.5021, -0.2642, -0.0116,  0.4408, -0.2803,  0.0456, -0.3797, -0.1006,
         0.0678,  0.5890,  0.3017, -0.8847,  0.3578, -0.1197,  0.5848,  0.1214,
         0.4657,  0.4148, -0.1520, -1.1701,  0.4323, -0.1848,  0.0093,  0.1662,
        -0.2472,  0.1840,  1.0480,  0.0881,  2.2527,  1.4496,  2.0630,  2.3970],
       grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 tensor([[1.9948, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.9948, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.9948,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 1.9948, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.9948, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9948]]),
 0,
 0,
 tensor([[1.1173, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.1173, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.1173,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 1.1173, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.1173, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.1173]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 tensor([[1.1013, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 1.1013, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [

In [25]:
projected_weights[0]

[0,
 tensor([ 0.2548,  0.0658, -0.0045,  0.0911, -0.0738,  0.0299,  0.0360,  0.1597,
          0.0923,  0.2516, -0.0078,  0.5584, -0.6457,  0.6559, -0.0656, -0.6471,
         -0.1306, -0.1466,  0.6234,  0.5314,  0.2881, -0.4003,  0.2266,  0.4650,
          0.3858,  0.4165, -0.1101, -0.1303,  0.5773, -0.4849,  0.2523, -0.5749,
         -0.1249, -0.1640, -0.0380, -0.3655,  0.8216,  0.3899,  0.0842, -0.5236]),
 0,
 0,
 tensor([ 3.5971e-01, -3.1273e-01,  9.5203e-02,  1.8245e-01,  1.4223e-01,
          2.6246e-01, -4.3272e-02, -1.8076e-01, -3.6497e-01, -2.8299e-01,
          1.3521e-01, -1.3890e-01,  3.6521e-02,  1.7641e-01,  3.2532e-01,
         -2.6646e-01,  3.6708e-01,  1.8529e-01, -7.7722e-02,  2.5398e-01,
         -3.6904e-04,  1.2581e-01,  3.6968e-01,  1.1169e-01, -5.0109e-02,
          6.3174e-02,  2.3317e-01,  2.0360e-02, -2.5412e-01, -3.3120e-01,
         -1.1161e-01,  3.6355e-01,  3.1893e-01, -3.2759e-01,  1.6140e-01,
          3.3690e-01, -5.5159e-01,  3.2794e-01,  5.5532e-01,  3

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
np.save( 'training_loss_dfedu' + str(eta).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), test_loss)
#np.save('relative_error_dfedu' + str(eta).replace('.', '_'), total_rel_error)